In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import ast



In [12]:
from textblob import TextBlob
import pyarrow

In [3]:
df = pd.read_json('/Users/danielh/Desktop/PF_EQ5_GOOGLE_YELP/PF_EQ5_GOOGLE_YELP/review.json', orient='records', lines=True, convert_dates=True)
df.tail()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27
6990279,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47


In [5]:
def analizar_sentimiento(text):
    sentimiento = TextBlob(text)
    return sentimiento.sentiment.polarity
       

In [6]:
df['text'] =df['text'].astype(str)
df['Polaridad del sentimiento'] =df['text'].apply(analizar_sentimiento)
df['Sentimiento'] =df['Polaridad del sentimiento'].apply(
    lambda x: 2 if x > 0 else (1 if x == 0 else 0)
)

In [29]:
df0 = df.drop(columns=['text', 'review_id'])
df0.head()

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,2018-07-07 22:09:11,0.085278,2
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,2012-01-03 15:28:18,0.402273,2
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,2014-02-05 20:30:30,0.139935,2
3,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,2015-01-04 00:01:03,0.302557,2
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,2017-01-14 20:54:15,0.400969,2


In [30]:
df0.to_parquet('review_sentiment_a.parquet')

In [10]:
dfb = pd.read_csv('business_id_final.csv')

In [11]:
dfb.head(5)

,0qu0fNTOsSmuREYVIMPuIQ
0,18eWJFJbXyR9j_5xfcRLYA
1,1FURjeGJi_LBXcJQg8eskw
2,1J0NIfmrSuV72KzhmYFxMA
3,1U8Kt44FIkC7RnJsNjaMTA
4,2pBNVsJo3a3CPbInUu9lvQ


In [31]:
df1 = pd.read_parquet('review_sentiment_a.parquet')

In [32]:
df1m =df1[['stars', 'business_id']].groupby(['business_id']).mean()
df1m

,stars
business_id,
---kPU91CF4Lq2-WlRu9Lw,4.500000
--0iUa4sNDFiZFrAdIWhZQ,3.214286
--30_8IhuyMHbSOcNWd6DQ,3.555556
--7PUidqRWpRSpXebiyxTg,1.750000
--7jw19RH9JKXgFohspgQw,4.230769
...,...
zznZqH9CiAznbkV6fXyHWA,4.916667
zztOG2cKm87I6Iw_tleZsQ,4.833333
zzu6_r3DxBJuXcjnOYVdTw,3.625000


In [33]:
df1u =df1[['useful', 'business_id']].groupby(['business_id']).mean()
df1u

,useful
business_id,
---kPU91CF4Lq2-WlRu9Lw,0.666667
--0iUa4sNDFiZFrAdIWhZQ,1.785714
--30_8IhuyMHbSOcNWd6DQ,0.777778
--7PUidqRWpRSpXebiyxTg,0.750000
--7jw19RH9JKXgFohspgQw,1.230769
...,...
zznZqH9CiAznbkV6fXyHWA,1.916667
zztOG2cKm87I6Iw_tleZsQ,0.166667
zzu6_r3DxBJuXcjnOYVdTw,1.125000


In [34]:
df1f =df1[['funny', 'business_id']].groupby(['business_id']).mean()
df1f

,funny
business_id,
---kPU91CF4Lq2-WlRu9Lw,0.041667
--0iUa4sNDFiZFrAdIWhZQ,0.214286
--30_8IhuyMHbSOcNWd6DQ,0.222222
--7PUidqRWpRSpXebiyxTg,0.166667
--7jw19RH9JKXgFohspgQw,0.000000
...,...
zznZqH9CiAznbkV6fXyHWA,0.250000
zztOG2cKm87I6Iw_tleZsQ,0.000000
zzu6_r3DxBJuXcjnOYVdTw,0.000000


In [35]:
df1c =df1[['cool', 'business_id']].groupby(['business_id']).mean()
df1c

,cool
business_id,
---kPU91CF4Lq2-WlRu9Lw,0.541667
--0iUa4sNDFiZFrAdIWhZQ,0.285714
--30_8IhuyMHbSOcNWd6DQ,0.000000
--7PUidqRWpRSpXebiyxTg,0.000000
--7jw19RH9JKXgFohspgQw,0.000000
...,...
zznZqH9CiAznbkV6fXyHWA,1.416667
zztOG2cKm87I6Iw_tleZsQ,0.333333
zzu6_r3DxBJuXcjnOYVdTw,0.375000


In [36]:
df1['ano'] = df1['date'].dt.year
df1['mes'] = df1['date'].dt.month

In [37]:
df1

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento,ano,mes
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,2018-07-07 22:09:11,0.085278,2,2018,7
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,2012-01-03 15:28:18,0.402273,2,2012,1
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,2014-02-05 20:30:30,0.139935,2,2014,2
3,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,2015-01-04 00:01:03,0.302557,2,2015,1
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,2017-01-14 20:54:15,0.400969,2,2017,1
...,...,...,...,...,...,...,...,...,...,...,...
6990275,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,2014-12-17 21:45:20,0.250000,2,2014,12
6990276,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,2021-03-31 16:55:10,0.422917,2,2021,3
6990277,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,2019-12-30 03:56:30,0.429167,2,2019,12
6990278,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,2022-01-19 18:59:27,0.311665,2,2022,1


In [40]:
df2020=df1[df1['ano'] == 2020]
df2020

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento,ano,mes
265179,IlWLPCRQp8iqX0X-ExQccQ,bdfZdB2MTXlT6-RBjSIpQg,4,0,0,0,2020-01-26 21:21:04,0.800000,2,2020,1
265180,nfLL3vn_4-zXAfoh7hWiZg,MbzgGsMQpGyVrUJXi_Jw0Q,5,0,0,1,2020-01-08 07:25:10,0.271354,2,2020,1
267425,kV4eLnU5fm-ieJCEeqNiIQ,ZY2feSm2sc5nTXYHnKpMGQ,5,0,0,0,2020-01-06 02:31:08,0.297222,2,2020,1
276825,sWVVqJQAl6w6cOzbUsUC3A,tsx84z4c0B-y6J5fqfvBqg,5,0,0,0,2020-01-15 12:59:01,0.513492,2,2020,1
276826,hflEbENcF1_TC2IoyX_Y4w,q8ZC6BgKh19RkLmH8bJ_Ig,5,0,0,0,2020-01-04 00:51:40,0.409714,2,2020,1
...,...,...,...,...,...,...,...,...,...,...,...
6990229,K37EM2POKtBRUum7M_d5hA,3CGXa85uNgvp8Berx5AhQQ,5,1,0,0,2020-03-10 04:52:29,0.352083,2,2020,3
6990230,jM9eD_bmcthFWliBUnWSLA,IoRTm7IAJqBbHpiisUBnLg,5,1,1,2,2020-07-30 18:36:12,0.386310,2,2020,7
6990233,6jjHo9Lilv3kTy87pm2ycw,pQAQwhBlSQdG1HuuLuCqXw,5,46,17,45,2020-10-11 00:09:30,0.248284,2,2020,10
6990235,gkg9VqsxPCgpfYXO1dl8CA,Ea663rIHyKXz2VP2DPH7Cg,4,3,0,0,2020-01-13 04:21:38,0.343182,2,2020,1


In [45]:
df2m =df2020[['stars', 'business_id', 'useful', 'funny', 'cool']].groupby(['business_id']).mean()
df2m

,stars,useful,funny,cool
business_id,,,,
---kPU91CF4Lq2-WlRu9Lw,4.583333,1.000000,0.000000,0.750000
--9osgUCSDUWUkoTLdvYhQ,2.000000,0.000000,0.000000,0.000000
--FcbSxK1AoEtEAxOgBaCw,4.200000,0.400000,0.000000,0.400000
--LC8cIrALInl2vyo701tg,5.000000,0.000000,0.000000,0.000000
--N9yp3ZWqQIm7DqKRvorg,1.000000,2.000000,0.000000,0.000000
...,...,...,...,...
zzjFdJwXuxBOGe9JeY_EMw,3.666667,0.888889,0.000000,0.222222
zznJox6-nmXlGYNWgTDwQQ,1.000000,0.000000,0.000000,0.000000
zznZqH9CiAznbkV6fXyHWA,4.909091,2.090909,0.272727,1.545455


In [42]:
df2021=df1[df1['ano'] == 2021]
df2021

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento,ano,mes
338961,iAD32p6h32eKDVxsPHSRHA,YB26JvvGS2LgkxEKOObSAw,5,0,0,0,2021-01-08 01:49:36,0.356994,2,2021,1
341768,rYvWv-Ny16b1lMcw1IP7JQ,jfIwOEXcVRyhZjM4ISOh4g,1,0,0,0,2021-01-02 00:19:00,-0.073333,0,2021,1
343516,v7i4M7NIx3bMNMChaXjU7Q,raKflkp3CANr8N7qpQ3ZyQ,5,0,0,0,2021-02-02 18:14:15,0.348810,2,2021,2
344290,2ntnbUia9Bna62W0fqNcxg,S-VD26LE_LeJNx5nASk_pw,5,0,0,0,2021-01-26 18:01:45,0.483333,2,2021,1
347458,j4qNLF-VNRF2DwBkUENW-w,yE1raqkLX7OZsjmX3qKIKg,5,0,0,0,2021-01-27 23:28:03,0.123333,2,2021,1
...,...,...,...,...,...,...,...,...,...,...,...
6990242,l5tQQASvVmkAIGNRa9PGDg,iYd-OZDRt0MHa6VWQkBUQA,5,0,0,0,2021-03-08 08:30:22,0.332500,2,2021,3
6990244,weNjiH8wxEF0xrMtceGQcg,q2sbzd5Z7Bi8pkmCyr-Umw,5,1,0,0,2021-10-05 18:51:55,0.151516,2,2021,10
6990253,XJTO9x78TgWE94cmXqNduA,c3QxX3toWdqJnKQmmIliRQ,1,1,2,0,2021-11-29 18:26:40,-0.186667,0,2021,11
6990257,5d32aJe0a3_fSl3LIVYkWQ,EmrOCT7fArKI0JI_XF6aaw,1,5,2,0,2021-08-28 18:46:05,-0.500000,0,2021,8


In [46]:
df3m =df2021[['stars', 'business_id', 'useful', 'funny', 'cool']].groupby(['business_id']).mean()
df3m

,stars,useful,funny,cool
business_id,,,,
---kPU91CF4Lq2-WlRu9Lw,4.363636,0.363636,0.090909,0.363636
--0iUa4sNDFiZFrAdIWhZQ,5.000000,0.000000,0.000000,0.000000
--9osgUCSDUWUkoTLdvYhQ,5.000000,0.250000,0.500000,0.000000
--FcbSxK1AoEtEAxOgBaCw,2.285714,0.571429,0.000000,0.142857
--N9yp3ZWqQIm7DqKRvorg,1.000000,0.500000,0.000000,0.000000
...,...,...,...,...
zziDpuuJw-Km1J4BaGpBKA,4.000000,1.000000,0.333333,1.000000
zzjFdJwXuxBOGe9JeY_EMw,3.600000,0.200000,0.200000,0.200000
zznJox6-nmXlGYNWgTDwQQ,1.000000,0.000000,0.000000,0.000000


In [41]:
df2022=df1[df1['ano'] == 2022]
df2022

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento,ano,mes
430101,BLu9dc1uj_MBgR-Ns9bwQg,drTZrkbpSoJgwKETlFbc3w,1,0,0,0,2022-01-01 15:47:07,0.275795,2,2022,1
430102,0w1Cpzqg0LV93LmrWbmZnA,jyxHti29yWdYR00Itt1A2w,5,0,0,0,2022-01-02 03:49:01,0.446429,2,2022,1
430105,QcP1iT3zKu7NQmiIlOg6XA,Jo4ei-c-5H53IxZxAVf1jQ,5,0,0,0,2022-01-03 03:17:03,0.400000,2,2022,1
430687,zH1VutqglmJPSvShRl07vg,YT5CjacTllBtvMaMJS3IbA,1,0,0,0,2022-01-05 15:55:59,-0.115079,0,2022,1
432403,bvbmmVvkoxzTFzPc89WQhA,9MHe5jAym2d8VhT_NbCRyw,2,0,0,0,2022-01-06 03:59:21,-0.273333,0,2022,1
...,...,...,...,...,...,...,...,...,...,...,...
6990266,cTozFTTWjlFYc3yusdbZmA,uMVOtr16r1ELu46pWr4HCQ,1,0,0,0,2022-01-18 06:42:59,-0.103947,0,2022,1
6990268,s67G457QlHSvk5RjOMN91w,58MJvmfo5hyfBbvkr54sFA,5,1,0,0,2022-01-18 15:24:44,0.357500,2,2022,1
6990272,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5,1,0,1,2022-01-17 22:36:01,0.568333,2,2022,1
6990273,_XVdmFWSgTN6YlojUxixTA,6WaI-IN8ql0xpEKlb4q8tg,5,1,0,0,2022-01-17 20:59:01,0.292857,2,2022,1


In [48]:
df3m =df2022[['stars', 'business_id', 'useful', 'funny', 'cool']].groupby(['business_id']).mean()
df3m

,stars,useful,funny,cool
business_id,,,,
---kPU91CF4Lq2-WlRu9Lw,5.0,0.000000,0.000000,0.000000
--9osgUCSDUWUkoTLdvYhQ,5.0,1.000000,0.000000,1.000000
--O3ip9NpXTKD4oBS1pY2A,5.0,0.500000,0.500000,1.000000
--S43ruInmIsGrnnkmavRw,2.0,0.000000,0.000000,0.000000
--hF_3v1JmU9nlu4zfXJ8Q,5.0,0.500000,0.000000,2.000000
...,...,...,...,...
zzIF9qp2UoHN48EeZH_IDg,4.0,0.000000,0.000000,0.000000
zzQWjZ_1Dr7kkDYlk17qRw,4.0,0.000000,0.000000,0.000000
zzXDi0Pdv0s84M-oQaIa_g,5.0,1.333333,0.666667,1.666667


In [53]:
df3stars=df3m[df3m['stars'] == 5.0]
df3stars['stars'].count()

7973

In [61]:
dfb.rename(columns={'0qu0fNTOsSmuREYVIMPuIQ':'business_id'}, inplace = True)

In [82]:
dfb=dfb.drop(columns=['a'])

In [83]:
dataframe_filtrado = pd.merge(df1, dfb, on='business_id', how='inner')
dataframe_filtrado

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento,ano,mes
0,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,2016-07-25 07:31:06,0.186364,2,2016,7
1,UsBxLh14sUpO8SdeqIiGOA,Wy8Hswf2cLQGRZN6armkag,1,1,0,0,2011-08-24 23:07:08,-0.078571,0,2011,8
2,3inG_FUhm28tUJc0zZ2fCg,aY_n9RSaD2Yw09jSFFePew,1,0,0,0,2017-08-16 15:43:19,-0.325000,0,2017,8
3,C_2mNjl-doRVvsL03_T57Q,18eWJFJbXyR9j_5xfcRLYA,4,7,0,0,2011-10-28 03:43:05,0.350000,2,2011,10
4,cpp89UW0imv45y3a_O444w,34Eqv8jXgxg_EEwcsNgeeg,5,0,0,0,2014-05-25 22:33:02,0.314524,2,2014,5
...,...,...,...,...,...,...,...,...,...,...,...
119387,XIkX0MgnhndkqVNQGOK4ig,KSYONgGtrK0nKXfroB-bwg,2,4,4,0,2021-10-21 06:04:46,0.007143,2,2021,10
119388,epj4YwV5NmNwZJvDSW0VGg,GuzbBFraIq-fbkjfvaTRvg,5,4,1,1,2021-12-15 00:47:29,0.231250,2,2021,12
119389,ut4UutJ_wDVkSaJtf9Ic3A,iT78kOB6Ja3BJQdYipPa5g,5,0,0,0,2021-11-24 03:49:21,0.642857,2,2021,11
119390,26_8S7BIL6YRw2PcS99Tnw,fiQxkb1pfPpvrr0oYYYBcA,5,0,0,0,2021-12-06 22:49:57,0.459444,2,2021,12


In [84]:
dataframe_filtrado.to_parquet('reviews_california.parquet')

In [85]:
dfrev_c = pd.read_parquet('reviews_california.parquet')
dfrev_c.head(5)

,user_id,business_id,stars,useful,funny,cool,date,Polaridad del sentimiento,Sentimiento,ano,mes
0,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,2016-07-25 07:31:06,0.186364,2,2016,7
1,UsBxLh14sUpO8SdeqIiGOA,Wy8Hswf2cLQGRZN6armkag,1,1,0,0,2011-08-24 23:07:08,-0.078571,0,2011,8
2,3inG_FUhm28tUJc0zZ2fCg,aY_n9RSaD2Yw09jSFFePew,1,0,0,0,2017-08-16 15:43:19,-0.325000,0,2017,8
3,C_2mNjl-doRVvsL03_T57Q,18eWJFJbXyR9j_5xfcRLYA,4,7,0,0,2011-10-28 03:43:05,0.350000,2,2011,10
4,cpp89UW0imv45y3a_O444w,34Eqv8jXgxg_EEwcsNgeeg,5,0,0,0,2014-05-25 22:33:02,0.314524,2,2014,5
